## Scrape a website and create a fairly tale type using the instruct embedding

In [ ]:
import sys
import pandas as pd

sys.path.append("../../../")

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from monologue.entities.examples import   AbstractVectorStoreEntry , INSTRUCT_EMBEDDING_VECTOR_LENGTH, Field, Optional, List, AbstractEntity
from monologue.core.data.stores import ColumnarDataStore, VectorDataStore

"""
define a type
"""
class FairyTales(AbstractVectorStoreEntry):
    class Config:
        embeddings_provider = "instruct"

    vector: Optional[List[float]] = Field(
        fixed_size_length=INSTRUCT_EMBEDDING_VECTOR_LENGTH
    )
        
        
"""
scrape some data into the type
"""
URL = "https://www.gutenberg.org/files/20748/20748-h/20748-h.htm"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")
elements = soup.find_all(lambda tag: tag.name in ['h2', 'p'])

records = []
current = None
store_index = 0
part_index = 0
for element in elements:
    #track header and decide what to do 
    if element.name == 'h2':
        if ']' in element.text:
            name = element.text.split(']')[-1]
            current = name
            store_index += 1
            part_index = 0
    elif current and element.text:
        part_index += 1
        key = name.replace(' ','-') + '-' + str(part_index)
        if len(element.text) > 50:
            ft = FairyTales(name=key, text=element.text)
            records.append(ft)
    #build an entry for generic database insert


In [ ]:
"""
create the store and populate it
"""
vstore = VectorDataStore(FairyTales)
#ds = vstore.add(records)

In [ ]:
df = vstore.run_search("Why did beauty suspect the beast did not want to harm her and why did the beast not want to let Beauty go vist her father?", probes=50)[['id', 'text', '_distance']]
df#.iloc[-1]['text']

In [ ]:
from monologue.core.agents.utils import summarize_data

summarize_data(df.to_dict('records'))

In [ ]:
#store("Who is aladdin and who is aladdins closest friend?")
#store("Where did alladins mother live?")
#store("What did the matchstick girl's grandmother tell her?")
#store("Where was the matchstick girl after she kindle a third match?")
store("Why did beauty suspect the beast did not want to harm her and why did the beast not want to let Beauty go vist her father?")


In [ ]:
#!pip install tantivy

# ask ChatGTP to create a dataset for some characters in the stories
(not shown - dataset provided in datasets)

In [ ]:
data = pd.read_feather('/Users/sirsh/Downloads/ft_characters.feather')
def as_list(x):
    try:
        return list(x)
    except:
        return None
data['closest_friends'] = data['closest_friends'].map(as_list)
data.to_feather('/Users/sirsh/Downloads/fairly_tail_characters.feather')
sample = data.to_dict('records')[0]
sample

In [ ]:
#use GPT to create a pydantic type for this
# from monologue.core.agents.utils import pydantic_type_generator
# pydantic_type_generator(sample, file_out='/Users/sirsh/Downloads/sample.fairy.py')

## Make a type and used the sample data and add it to the columnar store

In [ ]:
from typing import List
from pydantic import BaseModel

class FairyTaleCharacter(AbstractEntity):
    #add the key field
    character_name: str = Field(is_key=True)
    stature: str
    hair_color: str
    skin_color: str
    eye_color: str
    clothing_description: str
    closest_friends: List[str]
    arch_enemy: str
    times_left_home_or_country: float
    times_been_inside_castle: float
    story_appeared_in: str
    how_many_friends_they_have: float
    how_many_enemies_they_have: float
    how_many_animals_they_know: float


In [ ]:
 
# ft_chars = [FairyTaleCharacter(**sample) for sample in  data.to_dict('records')]
# ColumnarDataStore(FairyTaleCharacter).add(ft_chars)

In [ ]:
cstore = ColumnarDataStore(FairyTaleCharacter)

In [ ]:
#store.load()

In [ ]:
cstore("Who has spent the most time in a castle")

In [ ]:
cstore("Who has left home most often")

In [ ]:
cstore("Who is the most social character")

In [ ]:
cstore("Who are snow whites closest friends?")

In [ ]:
cstore("Who has the most interesting dress sense?")

In [ ]:
cstore("Name the story with the most characters that you know and list the characters?")

# How this works with open ai functions

In [ ]:
from monologue.core.agents.basic import call_open_ai_with_functions as the_program


In [ ]:
functions = [cstore.as_function, vstore.as_function]

In [ ]:
the_program(functions, "which character has the most friends and where did they eventually settle down")